In [1]:
import numpy as np
import hashlib
import json
import zipfile
import os
import glob
import shutil
print('Run this cell to ensure you have all the packages you need for the following program. Put the zip file full of txt in the same folder as this program.')

Run this cell to ensure you have all the packages you need for the following program. Put the zip file full of txt in the same folder as this program.


In [2]:
def dictionary_initialization_chat_and_transcript(csv_file):
    name_onyen_dictionary = {}
    with open(csv_file) as f:
        next(f)
        lines = f.readlines()
    lines = np.array(lines)
    for rows in lines:
        rows = rows.replace("\n", "")
        name_onyen_dictionary.update({rows.split(",")[0]:rows.split(",")[1]})
    print(name_onyen_dictionary)
    return name_onyen_dictionary

In [3]:
def extract_and_hash_chat(txt_file, csv_file):
    # Establish the name-onyen dictionary
    name_onyen_dictionary = dictionary_initialization_chat_and_transcript(csv_file)
    with open(txt_file) as f:
        lines = f.readlines()
    lines = np.array(lines)
    answer = []
    hash_map = {}
    for rows in lines:
        print("Original message: " + rows)
        if "From" and ":" in rows:
            if "(Direct Message)" not in rows:
                # initializing substrings
                sub1 = "From"
                sub2 = " :"

                # getting index of substrings
                idx1 = rows.index(sub1)
                idx2 = rows.index(sub2)

                # length of substring 1 is added to
                # get string from next character
                name = rows[idx1 + len(sub1) + 1: idx2].lstrip().rstrip() # remove spaces
                
                # Using the dictionary to convert the names to corresponding onyens
                onyen = name_onyen_dictionary[name]

                # sha-256 encryption (hash the onyen, NOT the name)
                encoded = onyen.encode()
                result = hashlib.sha256(encoded)

                hashed_str = rows[:idx1 + len(sub1) + 1] + str(result.hexdigest()) + rows[idx2:]
                print("Hashed message: " + hashed_str)
                answer.append(hashed_str)
                hash_map.update({name + "(" + onyen + ")":result.hexdigest()})
            if "(Direct Message)" in rows:
                # initializing substrings
                sub1 = "From"
                sub2 = "to"
                sub3 = "to"
                sub4 = "(Direct Message) :"

                # getting index of substrings
                idx1 = rows.index(sub1)
                idx2 = rows.index(sub2)
                idx3 = rows.index(sub3)
                idx4 = rows.index(sub4)

                # length of substring 1 is added to
                # get string from next character
                name1 = rows[idx1 + len(sub1) + 1: idx2].lstrip().rstrip() # remove spaces
                name2 = rows[idx3 + len(sub3) + 1: idx4].lstrip().rstrip() # remove spaces
                
                # Using the dictionary to convert the names to corresponding onyens
                onyen1 = name_onyen_dictionary[name1]
                onyen2 = name_onyen_dictionary[name2]

                # sha-256 encryption (hash the onyen, NOT the name)
                encoded1 = onyen1.encode()
                result1 = hashlib.sha256(encoded1)
                encoded2 = onyen2.encode()
                result2 = hashlib.sha256(encoded2)

                hashed_str = rows[:idx1 + len(sub1) + 1] + str(result1.hexdigest()) + " " + rows[idx2:idx3 + len(sub3)] + " " + str(result2.hexdigest()) + rows[idx4:]
                print("Hashed message: " + hashed_str)
                answer.append(hashed_str)
                hash_map.update({name1 + "(" + onyen1 + ")":result1.hexdigest(), name2 + "(" + onyen2 + ")":result2.hexdigest()})
        else:
            answer.append(rows)
    # Save the hashed txt file
    with open(txt_file.split('.')[0] + "(hashed_chat).txt", 'w') as f:
            for line in answer:
                f.write(line)
    # Save the hashmap in json file
    with open(txt_file.split('.')[0] +'(HashTable_chat).json', 'w') as fp:
        json.dump(hash_map, fp)
    return answer, hash_map

In [4]:
def extract_and_hash_audio_transcript(vtt_file, csv_file):
    # Establish the name-onyen dictionary
    name_onyen_dictionary = dictionary_initialization_chat_and_transcript(csv_file)
    with open(vtt_file) as f:
        lines = f.readlines()
    lines = np.array(lines)
    answer = []
    hash_map = {}
    for rows in lines:
        print("Original message: " + rows)
        if ":" in rows and rows.lstrip().rstrip() != "WEBVTT" and rows.lstrip().rstrip() != "" and rows[0] not in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] and rows[2] != ":":
            # initializing substrings
            sub = ": "

            # getting index of substrings
            idx = rows.index(sub)

            # length of substring 1 is added to
            # get string from next character
            name = rows[0: idx].lstrip().rstrip() # remove spaces

            # Using the dictionary to convert the names to corresponding onyens
            onyen = name_onyen_dictionary[name]

            # sha-256 encryption (hash the onyen, NOT the name)
            encoded = onyen.encode()
            result = hashlib.sha256(encoded)

            hashed_str = str(result.hexdigest()) + rows[idx:]
            print("Hashed message: " + hashed_str)
            answer.append(hashed_str)
            hash_map.update({name + "(" + onyen + ")":result.hexdigest()})
        else:
            answer.append(rows)
    # Save the hashed txt file
    with open(vtt_file.split('.')[0] + "(hashed_audio_transcript).txt", 'w') as f:
            for line in answer:
                f.write(line)
    # Save the hashmap in json file
    with open(vtt_file.split('.')[0] +'(HashTable_audio_transcript).json', 'w') as fp:
        json.dump(hash_map, fp)
    return answer, hash_map

In [5]:
def anonymyzer(zipfilename, csv_file):
    cwd = os.getcwd()
    with zipfile.ZipFile(cwd+'\\'+zipfilename+'.zip', 'r') as zip_ref:
        zip_ref.extractall(cwd)
    file_list = os.listdir(cwd+'\\'+zipfilename)
    for i in file_list:
        if (('hashed' not in i) and ('HashTable' not in i) and ('.txt' in i)): #chat txt files
            extract_and_hash_chat(cwd+'\\'+zipfilename+'\\'+i, csv_file)
        if (('hashed' not in i) and ('HashTable' not in i) and ('.vtt' in i)): # audio transcript vtt files
            extract_and_hash_audio_transcript(cwd+'\\'+zipfilename+'\\'+i, csv_file)
    # move files in separate files
    file_list = os.listdir(cwd+'\\'+zipfilename)
    try:
        os.makedirs(cwd+'\\'+'hashedChats') # create destination directory, if needed (similar to mkdir -p)
        os.makedirs(cwd+'\\'+'hashedTranscript') # create destination directory, if needed (similar to mkdir -p)
    except OSError:
        # The directory already existed, nothing to do
        pass
    try:
        os.makedirs(cwd+'\\'+'hashTables(Chat)') # create destination directory, if needed (similar to mkdir -p)
        os.makedirs(cwd+'\\'+'hashTables(Audio Transcript)') # create destination directory, if needed (similar to mkdir -p)
    except OSError:
        # The directory already existed, nothing to do
        pass

    for j in file_list:
        if ('hashed_chat' in j):
            shutil.move(cwd+'\\'+zipfilename+'\\'+j,cwd+'\\'+'hashedChats')
        if ('hashed_audio_transcript' in j):
            shutil.move(cwd+'\\'+zipfilename+'\\'+j,cwd+'\\'+'hashedTranscript')
        if ('HashTable_chat' in j):
            shutil.move(cwd+'\\'+zipfilename+'\\'+j,cwd+'\\'+'hashTables(Chat)')
        if ('HashTable_audio_transcript' in j):
            shutil.move(cwd+'\\'+zipfilename+'\\'+j,cwd+'\\'+'hashTables(Audio Transcript)')

In [6]:
zipfilename = 'Chat + Transcript'  #change the zip file name here WITHOUT extension
csv_file = "grade(Chat + Transcript).csv" #change the zip file name here WITH extension
anonymyzer(zipfilename, csv_file)

{'Bowen Gu': 'gubowen2', 'Hao (Harry) Wang': 'harrywh', 'A1 B1': 'onyen1', 'A2 (he/him)': 'onyen2', 'A2(he/him)': 'onyen3'}
Original message: 18:52:22	 From  Bowen Gu   to   Hao (Harry) Wang(Direct Message) : Hello World

Hashed message: 18:52:22	 From 74422ae6562d795fae4719624dd5a640744bb89f0b964ab4d7673c6a3de5263e to aab47dbbc82e215e0a7bfb455d5014821c0ca5c0e7862cb4e2f53e5026c0fa64(Direct Message) : Hello World

Original message: 18:52:26	 From  Hao (Harry) Wang   to   Bowen Gu(Direct Message) : 213123

Hashed message: 18:52:26	 From aab47dbbc82e215e0a7bfb455d5014821c0ca5c0e7862cb4e2f53e5026c0fa64 to 74422ae6562d795fae4719624dd5a640744bb89f0b964ab4d7673c6a3de5263e(Direct Message) : 213123

Original message: 18:52:30	 From  Hao (Harry) Wang : 2312

Hashed message: 18:52:30	 From aab47dbbc82e215e0a7bfb455d5014821c0ca5c0e7862cb4e2f53e5026c0fa64 : 2312

Original message: 18:52:35	 From  Bowen Gu   to   Hao (Harry) Wang(Direct Message) : abc

Hashed message: 18:52:35	 From 74422ae6562d795f